In [ ]:
#---------------------------
# Student Management System
#---------------------------
#This program allows an authenticated user to:
#add, view, search, update, and delete student records
#generate a simple report of student grades
#log all user actions for accountability
#
#Data will be stored in plain text files to keep things simple
#the file-handling shown here is beginner-friendly
#authentication and CRUD (Create, Read, Update, and Delete) operations

import os        #checks file existence and file operations
import getpass   #for secure password input (hides typed characters)
import datetime  #for timestamps in activity logs

In [ ]:
#------------------------------ 
# File paths (all data will be stored here)
# ------------------------------ 
USER_FILE = "users.txt"         #stores usernames and passwords
STUDENT_FILE = "students.txt"   #stores student records
LOG_FILE = "activity_log.txt"   #stores a log of all user actions

In [ ]:
def log_activity(username, action): 
    """
    Log user actions with timestamp. This helps track who did what and when
    """ 
    with open(LOG_FILE, "a") as log: 
        log.write(f"{datetime.datetime.now()} - {username} - {action}\n") 

#-------------------------
#User Management Functions
#-------------------------
def load_users(): 
    """ 
    Load all users from USER_FILE into a dictionary.
    Format in file: username,password
    Returns: { 'user1': 'pass1', 'user2': 'pass2', ... }
    """ 
    users = {} 
    if os.path.exists(USER_FILE): 
        with open(USER_FILE, "r") as f: 
            for line in f: 
                if "," in line: 
                    uname, pwd = line.strip().split(",", 1) 
                    users[uname] = pwd 
    return users 

In [ ]:
def save_user(username, password): 
    """
    Save new user to USER_FILE. 
    Appends to the file so that existing users aren't overwritten.
    """ 
    with open(USER_FILE, "a") as f: 
        f.write(f"{username},{password}\n") 

In [ ]:
def authenticate(): 
    """
    Authenticate user login. Prompts user for username and password.
    Checks credentials against USER_FILE.
    Returns the username if successful, otherwise None.
    """ 
    users = load_users() 
    username = input("Enter Username: ") 
    password = getpass.getpass("Enter Password: ") 
 
    if username in users and users[username] == password: 
        log_activity(username, "Logged in") 
        print("\n✅ Login Successful!\n") 
        return username 
    else: 
        print("\n❌ Invalid credentials!\n") 
        return None 

In [ ]:
#----------------------------
#Student management functions
#----------------------------

def add_student(username): 
    """
    Adds a new student record to STUDENT_FILE.
    Format: roll, name, grade
    """
    roll = input("Enter Roll Number: ") 
    name = input("Enter Name: ") 
    grade = input("Enter Grade: ") 
 
    with open(STUDENT_FILE, "a") as f: 
        f.write(f"{roll},{name},{grade}\n") 
 
    log_activity(username, f"Added student {roll}") 
    print("✅ Student added successfully!")

In [ ]:
def view_students(): 
    """
    Display all student records in a readable format.
    """
    if not os.path.exists(STUDENT_FILE): 
        print("No records found.") 
        return 
 
    with open(STUDENT_FILE, "r") as f: 
        print("\n--- Student Records ---") 
        for line in f: 
            roll, name, grade = line.strip().split(",") 
            print(f"Roll: {roll} | Name: {name} | Grade: {grade}") 

In [ ]:
def search_student(): 
    """
    search student by roll number and display their details
    """
    roll_no = input("Enter Roll Number to search: ") 
    found = False 
 
    if os.path.exists(STUDENT_FILE): 
        with open(STUDENT_FILE, "r") as f: 
            for line in f: 
                # Skip empty or invalid lines 
                parts = line.strip().split(",") 
                if len(parts) != 3: 
                    continue 
                roll, name, grade = parts 
                if roll == roll_no: 
                    print(f"✅ Found: Roll: {roll} | Name: {name} | Grade: {grade}") 
                    found = True 
                    break 
 
    if not found:
                    print("❌ Student not found.")



In [ ]:
def update_student(username): 
    """
    update existing student name and grade by roll number
    """
    roll_no = input("Enter Roll Number to update: ") 
    students = [] 
    updated = False 
 
    if os.path.exists(STUDENT_FILE): 
        with open(STUDENT_FILE, "r") as f: 
            students = f.readlines() 
 
        with open(STUDENT_FILE, "w") as f: 
            for line in students: 
                roll, name, grade = line.strip().split(",") 
                if roll == roll_no: 
                    print(f"Current Data → Name: {name}, Grade: {grade}") 
                    new_name = input("Enter new Name: ") 
                    new_grade = input("Enter new Grade: ") 
                    f.write(f"{roll},{new_name},{new_grade}\n") 
                    updated = True 
                    log_activity(username, f"Updated student {roll}") 
                else: 
                    f.write(line) 
 
    if updated: 
        print("✅ Student record updated.") 
    else: 
        print("❌ Student not found.")

In [ ]:
def delete_student(username): 
    """
    Delete a student by roll number
    """
    roll_no = input("Enter Roll Number to delete: ") 
    students = [] 
    deleted = False 
 
    if os.path.exists(STUDENT_FILE): 
        with open(STUDENT_FILE, "r") as f: 
            students = f.readlines() 
 
        with open(STUDENT_FILE, "w") as f: 
            for line in students: 
                # Skip empty or invalid lines 
                parts = line.strip().split(",") 
                if len(parts) != 3: 
                    f.write(line)  # keep invalid lines unchanged 
                    continue 
 
                roll, name, grade = parts 
                if roll == roll_no: 
                    deleted = True 
                    # Log deletion 
                    log_activity(username, f"Deleted student {roll}") 
                else: 
                    f.write(line) 
 
    if deleted: 
        print("✅ Student record deleted.") 
    else: 
        print("❌ Student not found.")

In [ ]:
#--------------------
#Reporting
#--------------------

def generate_report(): 
    """
    This generates a simple report with:
    -total number of students
    -count of students per grade
    """
    if not os.path.exists(STUDENT_FILE): 
        print("No records available.") 
        return 
 
    total = 0 
    grades = {} 
    with open(STUDENT_FILE, "r") as f: 
        for line in f: 
            roll, name, grade = line.strip().split(",") 
            total += 1 
            grades[grade] = grades.get(grade, 0) + 1 
 
    print("\n📊 Student Report") 
    print(f"Total Students: {total}") 
    for g, count in grades.items(): 
        print(f"Grade {g}: {count} student(s)") 

In [14]:
#--------------------
#Main Program
#--------------------

def main(): 
    """
    The entry point of the program. It will handle:
    -first-time admin creation
    -user login
    -menu navigation
    """
    print("===== Student Management System =====") 
 
    # Ensure admin user exists 
    if not os.path.exists(USER_FILE): 
        print("No users found. Create an Admin account.") 
        uname = input("Set Admin Username: ") 
        pwd = getpass.getpass("Set Admin Password: ") 
        save_user(uname, pwd) 
        print("✅ Admin created! Please restart the program.") 
 
    username = None 
    while not username: 
        username = authenticate() 
 
    # Menu 
    while True: 
        print("\n--- Main Menu ---") 
        print("1. Add Student") 
        print("2. View All Students") 
        print("3. Search Student") 
        print("4. Update Student") 
        print("5. Delete Student") 
        print("6. Generate Report") 
        print("7. Logout & Exit") 
 
        choice = input("Enter choice: ") 
 
        if choice == "1": 
            add_student(username) 
        elif choice == "2": 
            view_students() 
        elif choice == "3": 
            search_student() 
        elif choice == "4": 
            update_student(username) 
        elif choice == "5": 
            delete_student(username) 
        elif choice == "6": 
            generate_report() 
        elif choice == "7": 
            log_activity(username, "Logged out") 
            print("👋 Goodbye!") 
            break 
        else: 
            print("❌ Invalid choice! Please try again.") 
 
if __name__ == "__main__": 
    main() 

===== Student Management System =====
No users found. Create an Admin account.


Set Admin Username:  admin
Set Admin Password:  ········


✅ Admin created! Please restart the program.


Enter Username:  admin
Enter Password:  ········



✅ Login Successful!


--- Main Menu ---
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Generate Report
7. Logout & Exit


...

✅ Student record deleted.

--- Main Menu ---
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Generate Report
7. Logout & Exit


Enter choice:  6



📊 Student Report
Total Students: 2
Grade A: 1 student(s)
Grade B: 1 student(s)

--- Main Menu ---
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Generate Report
7. Logout & Exit


Enter choice:  7


👋 Goodbye!
